In this bidirectional gru model ,the code idea has been taken from "LSTM donor choose data".<br>
In this  model what we will do is we will using glove embedding to create embedding layer and also tokenize and pad sequences the text and sentiment column. the target will be selected_text column but it will not be possible to predict selected_text directlyl ,but with bidirectional lstm model to predict selected_text so what we will do is we will come up with two new columns for target variable ,these two columns will show the start index and end index of selected_text on text column and then based on start and end index we will create  array of indices from start to end and the word having common in text and selected text will be assigned as 1 and uncommon words will be assigned as 0.<br>
for example:

text | selected_text | start_index | end_index |
:---: | :---: | :---: | :---: |
my boss is bullying me | bullying me | 3 | 4|
what interview leave me alone | leave me alone | 2 | 4|

array will be :<br>
y_1 = [0,0,1,1,0]<br>
y_2 = [0,1,1,1,0] and so on<br>

the length of array will be max length of sentence in text column,for eg in above two sentence thelength is 5 so array is also of length 5 where common words are assigned 1 and rest index as 0.

so you can see above "bullying me start from 3rd index in text column and end at 4th index in text column,so  now we will use two newly created columns as target variable and during prediction will replace index with words,<br>This is the **BiDirectional GRU  MODEL**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fasttext

     |████████████████████████████████| 68 kB 2.2 MB/s 
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3137427 sha256=03052faa0bbdf0fe565bc1f1326c43eb66829d7bc6d80430072b28dcdb61e62a
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [3]:
import warnings
warnings.filterwarnings("ignore")
import re
import os
import random
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.stats as st
import matplotlib as mpl
import matplotlib.patches as mpl_patches
import matplotlib.cm as cm
import matplotlib.lines as mlines
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')  
nltk.download('vader_lexicon')
import spacy
from collections import defaultdict,Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer  
import fasttext.util
tqdm.pandas()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,GRU,LSTM
from tensorflow.keras.models import Model
import keras.backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding,Dense,Dropout,Concatenate,Flatten,Input,GRU,BatchNormalization,Bidirectional,SpatialDropout1D,LSTM,LayerNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping,TensorBoard,ReduceLROnPlateau

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/encoder_decoder_model/train_preprocessed_final.csv')
df_train.head(2)

,textID,text,selected_text,sentiment
0,cb774db0d1,i`d have responded if i were going,i`d have responded if i were going,neutral
1,549e992a42,sooo sad i will miss you here in san diego,sooo sad,negative


In [5]:
df_test = pd.read_csv('/content/drive/MyDrive/encoder_decoder_model/test_preprocessed_final.csv')
df_test.head(2)

,textID,text,sentiment
0,f87dea47db,last session of the day,neutral
1,96d74cb729,shanghai is also really exciting precisely sky...,positive


In [6]:
print('The training data has a shape of : ',df_train.shape)
print('The test data has a shape of : ',df_test.shape)

The training data has a shape of :  (27468, 4)
The test data has a shape of :  (3533, 3)


In [7]:
def start_index(data):
    '''This function will find the starting index from text column where words from selected_text starts
    text == attribute1
    selected_text == attribute2'''
    attribute1 = data[0]
    attribute2 = data[1]
    attribute1 = attribute1.split()
    attribute2 = attribute2.split()
    end = attribute2[0]
    starting_index = attribute1.index(end)
    return starting_index
    

In [8]:
def end_index(data):
    '''This function will find the ending index from text column where words from selected_text starts
    text == attribute1
    selected_text == attribute2
    start_index == attribute3'''
    attribute1 = data[0]
    attribute2 = data[1]
    attribute3 = data[2]
    attribute2 = attribute2.split()
    end = attribute2[-1]
    try:
        ending_index = attribute1.index(end,attribute3)
    except:
        ending_index = attribute1.index(end)
        
    return ending_index
    

In [9]:
df_train['starting_index'] = df_train[['text','selected_text']].progress_apply(lambda i : start_index(i),axis=1)

100%|██████████| 27468/27468 [00:00<00:00, 32697.11it/s]


In [10]:
df_train['ending_index'] = df_train[['text','selected_text','starting_index']].progress_apply(lambda i : end_index(i),axis=1)

100%|██████████| 27468/27468 [00:00<00:00, 47352.15it/s]


#### checking whether ending index < starting index and if yes removing that row

In [11]:
df_train[df_train.ending_index<df_train.starting_index].shape

(23, 6)

In [12]:
# we will take only those rows where ending_index >= starting index
df_train = df_train[df_train.ending_index >= df_train.starting_index]

In [13]:
df_train.shape

(27445, 6)

In [14]:
df_train.head()

,textID,text,selected_text,sentiment,starting_index,ending_index
0,cb774db0d1,i`d have responded if i were going,i`d have responded if i were going,neutral,0,6
1,549e992a42,sooo sad i will miss you here in san diego,sooo sad,negative,0,1
2,088c60f138,my boss is bullying me,bullying me,negative,3,4
3,9642c003ef,what interview leave me alone,leave me alone,negative,2,4
4,358bd9e861,sons of CURSE why couldn`t they put them on th...,sons of CURSE,negative,0,2


## Train Validation Split

#### Finding maximum length of sentence in text column from original data

In [15]:
df_train['text_word_length'] = df_train['text'].astype('str').apply(lambda i : len(i.split()))
max_sentence_length = df_train['text_word_length'].max()
print('The maximum length of sentence in text column is : ',max_sentence_length)

The maximum length of sentence in text column is :  32


In [16]:
X = df_train[["text","selected_text","sentiment","starting_index","ending_index"]]

In [17]:
y = np.zeros((X.shape[0],max_sentence_length+1))
for i,j in tqdm(enumerate(X.values)):
    ini_start_index = j[3] 
    fin_end_index = j[4]
    y[i][ini_start_index : fin_end_index+1] = 1

27445it [00:00, 463077.27it/s]


In [18]:
# Check the output y 
import random
for i in range(2):
    index = random.randint(0,df_train.shape[0])
    print("original data : ",X.values[index])
    print("\n")
    print("the output y :",y[index])
    print("*"*100)

original data :  ['i got a messgae but it didnt work cause i have a text saver not like money credit im sad now wat do i do xoxo'
 'sad' 'negative' 19 19]


the output y : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
****************************************************************************************************
original data :  ['feel crappy today' 'feel crappy today' 'negative' 0 2]


the output y : [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
****************************************************************************************************


#### Observation:
From above output you can see the the words at selected text column which are common to text column are assigned as 1 rest as 0 and the length of array is equal to meax length of sentence in text column which was designated by "max_sentence_length"

In [19]:
print('The X data has a shape of : ',X.shape)
print('The y data has a shape of : ',y.shape)

The X data has a shape of :  (27445, 5)
The y data has a shape of :  (27445, 33)


In [20]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.20,random_state=42)
#Preparing the y_train and y_valid from 2D to 3D tensor for seq to seq architecture
y_train=np.expand_dims(y_train,-1)
y_valid = np.expand_dims(y_valid,-1)
print('the shape of X_train is : ',X_train.shape)
print('the shape of X_valid is : ',X_valid.shape)
print('the shape of y_train is : ',y_train.shape)
print('the shape of y_valid is : ',y_valid.shape)

the shape of X_train is :  (21956, 5)
the shape of X_valid is :  (5489, 5)
the shape of y_train is :  (21956, 33, 1)
the shape of y_valid is :  (5489, 33, 1)


## "Text" column

In [21]:
df_text_train = X_train['text'].values
df_text_valid = X_valid['text'].values

#### Tokenizer and Pad sequencing

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
text_tknz = Tokenizer(lower=True,split=' ',oov_token='oov')
text_tknz.fit_on_texts(df_text_train)
df_text_train = text_tknz.texts_to_sequences(df_text_train)
df_text_valid = text_tknz.texts_to_sequences(df_text_valid)
text_vocab_size = len(text_tknz.word_index)+1
print(text_vocab_size)

20710


In [23]:
text_max_length = 32
from tensorflow.keras.preprocessing.sequence import pad_sequences
df_text_train = pad_sequences(df_text_train,maxlen=text_max_length,padding='post')
df_text_valid = pad_sequences(df_text_valid,maxlen=text_max_length,padding='post')

print('the shape of text train after padding : ',df_text_train.shape)
print('the shape of text valid after padding : ',df_text_valid.shape)

the shape of text train after padding :  (21956, 32)
the shape of text valid after padding :  (5489, 32)


#### Fasttext Emdedding Matrix
#https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

In [24]:
fasttext.util.download_model('en', if_exists='ignore')

'cc.en.300.bin'

In [25]:
fast_txt = fasttext.load_model('cc.en.300.bin')

In [26]:
embedding_matrix=np.zeros(((text_vocab_size,300)))
for word,i in text_tknz.word_index.items(): 
    if type(word)==bool:
        continue
    else:
        embedding_vector=fast_txt.get_word_vector(word)
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector
print(embedding_matrix.shape)

(20710, 300)


## "Sentiment" column

In [27]:
df_sentiment_train = X_train['sentiment'].values
df_sentiment_valid = X_valid['sentiment'].values

#### Tokenizer and Pad sequencing

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
sentiment_tknz = Tokenizer(lower=True,split=' ',oov_token='oov')
sentiment_tknz.fit_on_texts(df_sentiment_train)
df_sentiment_train = sentiment_tknz.texts_to_sequences(df_sentiment_train)
df_sentiment_valid = sentiment_tknz.texts_to_sequences(df_sentiment_valid)
sentiment_vocab_size = len(sentiment_tknz.word_index)+1
print(sentiment_vocab_size)

5


In [29]:
sentiment_max_length = 1
from tensorflow.keras.preprocessing.sequence import pad_sequences
df_sentiment_train = pad_sequences(df_sentiment_train,maxlen=sentiment_max_length,padding='post')
df_sentiment_valid = pad_sequences(df_sentiment_valid,maxlen=sentiment_max_length,padding='post')

print('the shape of sentiment train after padding : ',df_sentiment_train.shape)
print('the shape of sentiment valid after padding : ',df_sentiment_valid.shape)

the shape of sentiment train after padding :  (21956, 1)
the shape of sentiment valid after padding :  (5489, 1)


## Bidirectional LSTM Seq2seq model

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding,Dense,LSTM,Dropout,Concatenate,Flatten,TimeDistributed,Input,GRU,BatchNormalization,Bidirectional,SpatialDropout1D,LSTM,LayerNormalization
from tensorflow.keras.regularizers import l2

In [31]:
# https://machinelearningmastery.com/adding-a-custom-attention-layer-to-recurrent-neural-network-in-keras/

class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True):
        super().__init__()
        self.return_sequences = return_sequences    
    def build(self, input_shape):
        
        self.W=self.add_weight(name="attention_weight", shape=(input_shape[-1],1),
                               initializer="random_normal",trainable =True)
        self.b=self.add_weight(name="attention_bias", shape=(input_shape[1],1),
                               initializer="zeros",trainable=True)
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        # Alignment scores. Pass them through tanh function
        e = tanh(dot(x,self.W)+self.b)
        # Compute the weights
        alpha =  tf.nn.softmax(e, axis=1)
        ## Compute the context vector
        context = x*alpha
        if self.return_sequences:
            return context
        return  tf.reduce_sum(context, axis=1)

In [32]:
#https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/

tf.keras.backend.clear_session()
#Text column input
text_input = Input(shape=(text_max_length,))
#Sentiment column input
sentiment_input = Input(shape=(sentiment_max_length,))
#Concatinating both inputs
concat= Concatenate()([text_input,sentiment_input])
embedding_text = Embedding(text_vocab_size,300,input_length=text_max_length,\
                      trainable=False,mask_zero = True,embeddings_initializer=tf.constant_initializer(embedding_matrix))(concat)
bigru_layer = Bidirectional(GRU(256,return_sequences=True,dropout=0.4))(embedding_text)
attention_layer = attention(return_sequences=True)(bigru_layer)
dense_layer_1 = Dense(64,activation='relu',kernel_regularizer=l2(0.0001))(attention_layer)#(bilstm_layer)
dropout_layer = Dropout(0.25)(dense_layer_1)
dense_layer_2 = Dense(16,activation='relu',kernel_regularizer=l2(0.0001))(dropout_layer)
output=TimeDistributed(Dense(1,activation='sigmoid'))(dense_layer_2)

In [33]:
model=Model(inputs=[text_input,sentiment_input],outputs=[output])

In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 33)           0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 embedding (Embedding)          (None, 33, 300)      6213000     ['concatenate[0][0]']        

In [35]:
train_input = (df_text_train, df_sentiment_train)
train_output = y_train

valid_input = (df_text_valid,df_sentiment_valid)
valid_output = y_valid

In [36]:
## Custom loss function

## source : nlp_attention_mechanism assignment

def loss_function(real,pred):
    """ Custom loss function that will not consider the loss for padded zeros.
    why are we using this, can't we use simple binary crossentropy?
    Yes, you can use simple binary crossentropy , but in this loss function we are ignoring the loss
    for the padded zeros. i.e when the input is zero then we donot need to worry what the output is. This padded zeros are added from our end
    during preprocessing to make equal length for all the sentences.

    """
    loss_object = tf.keras.losses.BinaryCrossentropy(reduction = tf.keras.losses.Reduction.SUM)
    loss_ = loss_object(real,pred)
    return loss_ / 128

In [37]:
import datetime
import os
! mkdir  'checkpt11'
file_path = os.path.join('model111.hdf5')
earlystop = EarlyStopping(monitor='val_mae', min_delta=0.001, patience=10, verbose=1,mode='min')

reducelr = ReduceLROnPlateau(monitor='val_mae', min_delta=0.001, patience=5, verbose=1, factor=0.9)
checkpt_model = tf.keras.callbacks.ModelCheckpoint(filepath=file_path,save_weights_only=True,monitor='val_mae',save_best_only=True,verbose=1)
callbacks=[checkpt_model,earlystop,reducelr]

In [38]:
model.compile(optimizer='adam',loss=loss_function,metrics=['accuracy'])

In [39]:
model.fit(train_input,train_output,epochs=15,batch_size=128,validation_data=(valid_input,valid_output),callbacks=callbacks)

Epoch 1/15
172/172 [==============================] - 173s 748ms/step - loss: 10.8196 - accuracy: 0.9073 - val_loss: 5.8752 - val_accuracy: 0.9245
Epoch 2/15
172/172 [==============================] - 112s 651ms/step - loss: 5.6339 - accuracy: 0.9283 - val_loss: 5.3804 - val_accuracy: 0.9303
Epoch 3/15
172/172 [==============================] - 82s 475ms/step - loss: 5.3891 - accuracy: 0.9306 - val_loss: 5.2551 - val_accuracy: 0.9311
Epoch 4/15
172/172 [==============================] - 74s 430ms/step - loss: 5.2836 - accuracy: 0.9316 - val_loss: 5.1556 - val_accuracy: 0.9327
Epoch 5/15
172/172 [==============================] - 72s 419ms/step - loss: 5.1822 - accuracy: 0.9327 - val_loss: 5.1172 - val_accuracy: 0.9334
Epoch 6/15
172/172 [==============================] - 72s 420ms/step - loss: 5.1160 - accuracy: 0.9336 - val_loss: 5.1261 - val_accuracy: 0.9330
Epoch 7/15
172/172 [==============================] - 72s 416ms/step - loss: 5.0383 - accuracy: 0.9343 - val_loss: 5.0663 - val

## Jaccard-Score for Train Data

In [40]:
train_data_new = X_train[["text","selected_text","sentiment"]]
train_data_new.shape

(21956, 3)

In [41]:
y_train_prediction = model.predict(train_input)
y_train_prediction = np.squeeze(y_train_prediction)
y_train_prediction = np.round(y_train_prediction)
print('the shape of predicted data : ',y_train_prediction.shape)

the shape of predicted data :  (21956, 33)


In [42]:
## printing the predicted output as an array of index where common words present in text and selected_text
train_data_prediction = []
for i in tqdm(y_train_prediction):
    index = []
    for j in range(len(i)):
        if i[j] == 1:
            index.append(j)
        else:
            continue
    index = np.array(index)
    train_data_prediction.append(index)
print('The length of predicted list is : ',len(train_data_prediction))

100%|██████████| 21956/21956 [00:01<00:00, 13549.17it/s]

The length of predicted list is :  21956


In [43]:
train_data_new['prediction_arr'] = train_data_prediction
train_data_new.head(2)

,text,selected_text,sentiment,prediction_arr
9772,someone take me to la i need to see the lemon ...,someone take me to la i need to see the lemon ...,neutral,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
26215,lol heeeyyy love not much working and you,lol heeeyyy love not much working and you,neutral,"[0, 1, 2, 3, 4, 5, 6, 7]"


In [44]:
def converting_to_text(x):
    """ This function convert the respective index number to its corresponding words"""
    predicting_text  = []
    text_column = data[0].split()
    index = data[1]
    l = len(text_column)
    for i in index:
        if i < 1:
            predicting_text.append(text_column[i])
    return predicting_text

In [46]:
train_prediction = train_data_new[['text','prediction_arr']].progress_apply(lambda i : converting_to_text(i),axis=1)

100%|██████████| 21956/21956 [00:00<00:00, 27954.25it/s]


In [47]:
train_data_new['predicted_text'] = train_prediction
train_data_new['predicted_text'] = train_data_new['predicted_text'].apply(lambda i : ' '.join(i))
train_data_new.head(2)

,text,selected_text,sentiment,prediction_arr,predicted_text
9772,someone take me to la i need to see the lemon ...,someone take me to la i need to see the lemon ...,neutral,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",someone take me to la i need to see the lemon ...
26215,lol heeeyyy love not much working and you,lol heeeyyy love not much working and you,neutral,"[0, 1, 2, 3, 4, 5, 6, 7]",lol heeeyyy love not much working and you


In [48]:
def jaccard(x): 
    str1 = str(x[0])
    str2 = str(x[1])
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [49]:
train_data_new['jaccard_score'] = train_data_new[['selected_text','predicted_text']].progress_apply(jaccard,axis=1)

100%|██████████| 21956/21956 [00:00<00:00, 45456.81it/s]


In [50]:
print('Mean jaccard score for training data:', np.mean(train_data_new['jaccard_score']))

Mean jaccard score for training data: 0.6949667188453018


## Jaccard-Score on Validation Data

In [51]:
valid_data_new = X_valid[["text","selected_text","sentiment"]]
valid_data_new.shape

(5489, 3)

In [52]:
y_valid_prediction = model.predict(valid_input)
y_valid_prediction = np.squeeze(y_valid_prediction)
y_valid_prediction = np.round(y_valid_prediction)
print('the shape of predicted data : ',y_valid_prediction.shape)

the shape of predicted data :  (5489, 33)


In [53]:
## printing the predicted output as an array of index where common words present in text and selected_text
valid_data_prediction = []
for i in tqdm(y_valid_prediction):
    index = []
    for j in range(len(i)):
        if i[j] == 1:
            index.append(j)
        else:
            continue
    index = np.array(index)
    valid_data_prediction.append(index)
print('The length of predicted list is : ',len(valid_data_prediction))

100%|██████████| 5489/5489 [00:00<00:00, 12830.03it/s]

The length of predicted list is :  5489


In [54]:
valid_data_new['prediction_arr'] = valid_data_prediction
valid_data_new.head(2)

,text,selected_text,sentiment,prediction_arr
1366,sooo glad im home floridia was fun back in atl...,glad,positive,"[1, 6]"
11297,csi ny comes back to aus with ` CURSE lies and...,csi ny comes back to aus with ` CURSE lies and...,neutral,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [55]:
valid_prediction = valid_data_new[['text','prediction_arr']].progress_apply(lambda i : converting_to_text(i),axis=1)

100%|██████████| 5489/5489 [00:00<00:00, 39280.70it/s]


In [56]:
valid_data_new['predicted_text'] = valid_prediction
valid_data_new['predicted_text'] = valid_data_new['predicted_text'].apply(lambda i : ' '.join(i))
valid_data_new.head(2)

,text,selected_text,sentiment,prediction_arr,predicted_text
1366,sooo glad im home floridia was fun back in atl...,glad,positive,"[1, 6]",glad fun
11297,csi ny comes back to aus with ` CURSE lies and...,csi ny comes back to aus with ` CURSE lies and...,neutral,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",csi ny comes back to aus with ` CURSE lies and...


In [57]:
valid_data_new['jaccard_score'] = valid_data_new[['selected_text','predicted_text']].progress_apply(jaccard,axis=1)

100%|██████████| 5489/5489 [00:00<00:00, 46605.64it/s]


In [58]:
print('Mean jaccard score for validation data:', np.mean(valid_data_new['jaccard_score']))

Mean jaccard score for validation data: 0.661694544966673


## Predicting "selected_text" on Test Data

In [59]:
df_test.head(2)

,textID,text,sentiment
0,f87dea47db,last session of the day,neutral
1,96d74cb729,shanghai is also really exciting precisely sky...,positive


#### Tokenizing and pad sequencing Text Column

In [60]:
df_text_test = df_test['text'].values
df_text_test = text_tknz.texts_to_sequences(df_text_test)
df_text_test = pad_sequences(df_text_test,maxlen=text_max_length,padding='post')
print('the shape of test text : ',df_text_test.shape)


the shape of test text :  (3533, 32)


#### Tokenizing and pad sequencing sentiment column

In [61]:
df_sentiment_test = df_test['sentiment'].values
df_sentiment_test = sentiment_tknz.texts_to_sequences(df_sentiment_test)
df_sentiment_test = pad_sequences(df_sentiment_test,maxlen=sentiment_max_length,padding='post')
print('the shape of test sentiment : ',df_sentiment_test.shape)

the shape of test sentiment :  (3533, 1)


In [62]:
predicted_results  = model.predict([df_text_test,df_sentiment_test])

In [63]:
predicted_results.shape

(3533, 33, 1)

In [64]:
predicted_results = np.squeeze(predicted_results)
predicted_results = np.round(predicted_results)
print('the shape of predicted data : ',predicted_results.shape)

the shape of predicted data :  (3533, 33)


In [65]:
## printing the predicted output as an array of index where common words present in text and selected_text
test_data_prediction = []
for i in tqdm(predicted_results):
    index = []
    for j in range(len(i)):
        if i[j] == 1:
            index.append(j)
        else:
            continue
    index = np.array(index)
    test_data_prediction.append(index)
print('The length of predicted list is : ',len(test_data_prediction))

100%|██████████| 3533/3533 [00:00<00:00, 13120.92it/s]

The length of predicted list is :  3533


In [66]:
test_data = df_test.copy()

In [67]:
test_data['prediction_arr'] = test_data_prediction
test_data.head(2)

,textID,text,sentiment,prediction_arr
0,f87dea47db,last session of the day,neutral,"[0, 1, 2, 3, 4]"
1,96d74cb729,shanghai is also really exciting precisely sky...,positive,[4]


In [68]:
test_prediction = test_data[['text','prediction_arr']].progress_apply(lambda i : converting_to_text(i),axis=1)

100%|██████████| 3533/3533 [00:00<00:00, 41410.20it/s]


In [69]:
test_data['selected_text'] = test_prediction
test_data['selected_text'] = test_data['selected_text'].apply(lambda i : ' '.join(i))
test_data = test_data.drop('prediction_arr',axis=1)
test_data.head(5)

,textID,text,sentiment,selected_text
0,f87dea47db,last session of the day,neutral,last session of the day
1,96d74cb729,shanghai is also really exciting precisely sky...,positive,exciting
2,eee518ae67,recession hit veronique branquinho she has to ...,negative,a shame
3,01082688c6,happy bday,positive,happy
4,33987a8ee5,i like it,positive,i like it


### Observation/Result

In [1]:
# for Glove embedding result shown here please refer Bidirectional_lstm_seq2seq_model_fasttext_embedding.ipynb file

from prettytable import PrettyTable
result = PrettyTable()
result.field_names = ["Model","Train_Jaccard_score","Validation_Jaccard_score"]
result.add_row(["LSTM Base Model",0.5037,0.492])
result.add_row(["BiDirectional LSTM Seq2Seq Glove Embedding ",0.701,0.652])
result.add_row(["BiDirectional LSTM Seq2Seq FastText Embedding ",0.671,0.651])
result.add_row(["BiDirectional LSTM Seq2Seq attention model with Glove Embedding ",0.701,0.645])
result.add_row(["BiDirectional LSTM Seq2Seq attention model with FastText Embedding ",0.655,0.632])
result.add_row(["BiDirectional GRU Seq2Seq Glove Embedding ",0.692,0.645])
result.add_row(["BiDirectional GRU Seq2Seq FastText Embedding ",0.674,0.651])
result.add_row(["BiDirectional GRU Seq2Seq attention model with Glove Embedding ",0.724,0.656])
result.add_row(["BiDirectional GRU Seq2Seq attention model with Fast-Text Embedding ",0.694,0.661])
print(result)

+---------------------------------------------------------------------+---------------------+--------------------------+
|                                Model                                | Train_Jaccard_score | Validation_Jaccard_score |
+---------------------------------------------------------------------+---------------------+--------------------------+
|                           LSTM Base Model                           |        0.5037       |          0.492           |
|             BiDirectional LSTM Seq2Seq Glove Embedding              |        0.701        |          0.652           |
|            BiDirectional LSTM Seq2Seq FastText Embedding            |        0.671        |          0.651           |
|   BiDirectional LSTM Seq2Seq attention model with Glove Embedding   |        0.701        |          0.645           |
| BiDirectional LSTM Seq2Seq attention model with FastText Embedding  |        0.655        |          0.632           |
|              BiDirectional GRU

### Conclusion

We can see from above observation that adding attention layers on bidirectional gru seq2seq model with glove embedding with trying various nodes on layer has little improved the result whereas when using fasttext embedding the model with atention layer has a same result as fpr seq2seq model for the best selected nodes (after trying various nodes on architecture), so we would prefer bidirectional gru model with attention layer as its improved result on training the model.
